# Testing Neo4j graph on current BART model

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

import neo4j

import csv

from geographiclib.geodesic import Geodesic

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [6]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)

In [7]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [8]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [9]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [10]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [11]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [12]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [13]:
cursor = connection.cursor()

# Wipe out the Neo4j database

Call the function my_neo4j_wipe_out_database() to wipe out the Neo4j database

In [14]:
my_neo4j_wipe_out_database()

# Verify the number of nodes and relationships

In [15]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Find all zip codes, with population, within 1 mile of the Downtown Berkeley station

Our stations table has each station, along with the latitude and longitude point for the station

The function above my_station_get_zips() takes a station name and X miles (which can be decimal such as 1.5) and finds all zip codes whose geograpic center lies within a box X miles from the station along with the population and the total population

Zip code data is pulled from the zip_codes table

Week 2 has an optional module that covers geodesic distance calculation and how to use the box method




## Since this is the first one, a solution code cell is provided for you to execute and then pattern the rest after



In [16]:
my_station_get_zips('Downtown Berkeley', 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------


# Find all zip codes, with population, within 1, 2, 3, 4, and 5 miles of the Downtown Berkeley station

Write a loop to call the function my_station_get_zips() for each of 1, 2, 3, 4, and 5 miles




In [17]:
for miles in range(1,6):
    my_station_get_zips('Downtown Berkeley', miles)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94618   population:      17,041
     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   pop

# Louvain Modularity algorithm

In [18]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [19]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)

,name,community,intermediate_community


In [20]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId, intermediateCommunityIds
WITH gds.util.asNode(nodeId) AS node, communityId, intermediateCommunityIds
WHERE communityId = 10
RETURN node.name AS name, communityId AS community, intermediateCommunityIds AS intermediate_community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)


,name,community,intermediate_community


In [21]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD communityId
RETURN count(DISTINCT communityId) AS totalCommunities

"""

my_neo4j_run_query_pandas(query)

,totalCommunities
0,0


# Simplified graph for Louvain Modularity

In [22]:
my_neo4j_wipe_out_database()

In [23]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node(station)

# Verifying number of nodes and relationships

In [24]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 0
-------------------------


# Looking at query

In [25]:
rollback_before_flag = True
rollback_after_flag = True

query = """

(
select distinct a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where a.station < b.station
)
UNION
(
select distinct b.station as from_station, a.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where a.station > b.station
)
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,from_station,to_station,travel_time
0,12th Street,19th Street,120
1,12th Street,Lake Merritt,180
2,12th Street,West Oakland,300
3,16th Street Mission,24th Street Mission,120
4,16th Street Mission,Civic Center,180
5,19th Street,MacArthur,180
6,24th Street Mission,Glen Park,180
7,Antioch,Pittsburg Center,420
8,Ashby,Downtown Berkeley,180
9,Ashby,MacArthur,240


# Creating a relationship for each segment both ways

In [26]:
connection.rollback()

query = """

(
select distinct a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where a.station < b.station
)
UNION
(
select distinct b.station as from_station, a.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where a.station > b.station
)
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    from_station = row[0]
    to_station = row[1]
    travel_time = int(row[2])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

# Verifying the number of nodes and relationships

In [27]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 102
-------------------------


# Louvain Modularity algorithm on simplified graph

In [28]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [29]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)

,name,community,intermediate_community
0,Ashby,2,"[15, 2]"
1,Downtown Berkeley,2,"[15, 2]"
2,El Cerrito Plaza,2,"[38, 2]"
3,El Cerrito del Norte,2,"[38, 2]"
4,North Berkeley,2,"[15, 2]"
5,Richmond,2,"[38, 2]"
6,16th Street Mission,10,"[3, 10]"
7,24th Street Mission,10,"[3, 10]"
8,Civic Center,10,"[10, 10]"
9,Glen Park,10,"[3, 10]"


# How many communities

In [30]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD communityId
RETURN count(DISTINCT communityId) AS totalCommunities

"""

my_neo4j_run_query_pandas(query)

,totalCommunities
0,8


# 2.0 Create Capacity Table

## 2.1 Drop the capacity table if it exists

In [31]:
connection.rollback()

query = """

drop table if exists capacity;

"""

cursor.execute(query)

connection.commit()

## 2.2 Create the capacity table and capacity

In [32]:
connection.rollback()

query = """

create table capacity (
    weekday date,
    entry_station varchar(32),
    exit_station varchar(32),
    exits_count numeric(32),
    entry_station_full_name varchar(128),
    exit_station_full_name varchar(128)
);

"""

cursor.execute(query)

connection.commit()

In [33]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [34]:
my_read_csv_file("BART_entry_exist.csv", limit=10)

['weekday', 'entry_station', 'exit_station', 'exits count', 'entry_station_full_name', 'exit_station_full_name']
['2020-01-01 00:00:00', 'RM', 'RM', '19.857142857142900', 'Richmond', 'Richmond']
['2020-01-01 00:00:00', 'EN', 'RM', '119.52380952381000', 'El Cerrito Del Norte', 'Richmond']
['2020-01-01 00:00:00', 'EP', 'RM', '86.61904761904760', 'El Cerrito Plaza', 'Richmond']
['2020-01-01 00:00:00', 'NB', 'RM', '70.23809523809520', 'North Berkeley', 'Richmond']
['2020-01-01 00:00:00', 'BK', 'RM', '389.5238095238100', 'Berkeley', 'Richmond']
['2020-01-01 00:00:00', 'AS', 'RM', '93.47619047619050', 'Ashby', 'Richmond']
['2020-01-01 00:00:00', 'MA', 'RM', '140.0', 'MacArthur', 'Richmond']
['2020-01-01 00:00:00', '19', 'RM', '168.61904761904800', '19th Street Oakland', 'Richmond']
['2020-01-01 00:00:00', '12', 'RM', '183.04761904761900', '12th Street / Oakland City Center', 'Richmond']

Printed  10 lines of  105001 total lines.


## 2.3 Load capacity csv

In [35]:
connection.rollback()

query = """

copy capacity
from '/user/projects/project-3-gonz-ga-ds/code/BART_entry_exist.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

## 2.4 concat Communites with Capacity

In [36]:
# get capacity df
rollback_before_flag = True
rollback_after_flag = True

query = """

with entry_station_count as (
                            select
                                    avg(capacity.exits_count) as entry_count_month_avg,
                                    entry_station_full_name as station
                            from capacity
                            group by entry_station_full_name
                            ),
    exit_station_count as (
                            select
                                    avg(capacity.exits_count) as exit_count_month_avg,
                                    exit_station_full_name as station
                            from capacity
                            group by exit_station_full_name
                            )               

select 
        entry_station_count.station as station,
        entry_station_count.entry_count_month_avg,
        exit_station_count.exit_count_month_avg,
        (entry_station_count.entry_count_month_avg + exit_station_count.exit_count_month_avg) as total_monthly_capacity
from entry_station_count join exit_station_count on entry_station_count.station = exit_station_count.station
order by station;

"""

capacity_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# get community df
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

community_df = my_neo4j_run_query_pandas(query)

# Concatenate two df
supplimentary_list = []
def station_name_process(x):
    x = x.strip()
    if "/" in x:
        x1 = x[0:x.find("/")].strip()
        x2 = x[x.find("/")+1:len(x)].strip()
        supplimentary_list.append([x2,x1])
        return x1
    elif x == "Berkeley":
        return "Downtown Berkeley"
    elif x == "Oakland International Airport":
        return "OAK"
    elif x == "19th Street Oakland":
        return "19th Street"
    elif x == "San Francisco International Airport":
        return "SFO"
    else:
        return x

capacity_df["station"] = capacity_df["station"].apply(lambda x:station_name_process(x))


for i in supplimentary_list:
    append_list = capacity_df.loc[capacity_df["station"]==i[1]].values[0]
    append_list[0] = i[0]
    capacity_df.loc[len(capacity_df)+1] = append_list

community_df["station_key"] = community_df["name"].str.lower()
community_df["station_key"] = community_df["station_key"].apply(lambda x:x.replace(" ",""))
capacity_df["station_key"] = capacity_df["station"].str.lower()
capacity_df["station_key"] = capacity_df["station_key"].apply(lambda x:x.replace(" ",""))

community_capcity_df = pd.merge(left=community_df,right=capacity_df,on="station_key",how="left",indicator=False)

def capacity_community(community):
    temp_df = community_capcity_df.copy()
    temp_df = temp_df.loc[temp_df["community"]==community]
    temp_df = temp_df[["station","community","intermediate_community","entry_count_month_avg","exit_count_month_avg","total_monthly_capacity"]]
    print(temp_df.sort_values("total_monthly_capacity"))
    # show summary
    print("--> Monthly average total capacity:",temp_df["total_monthly_capacity"].sum())
    print("--> Monthly average entry capacity:",temp_df["entry_count_month_avg"].sum())
    print("--> Monthly average exit capacity:",temp_df["exit_count_month_avg"].sum())
    print("--> Top 3 capacity station in the community: \n",temp_df.sort_values("total_monthly_capacity").head(3))
    print("--> Top 3 entry capacity station in the community: \n",temp_df.sort_values("entry_count_month_avg").head(3))
    print("--> Top 3 exit station in the community: \n",temp_df.sort_values("exit_count_month_avg").head(3))

In [37]:
capacity_community(10)

                station  community intermediate_community  \
9             Glen Park         10                [3, 10]   
7   24th Street Mission         10                [3, 10]   
6   16th Street Mission         10                [3, 10]   
8          Civic Center         10               [10, 10]   
10        Powell Street         10               [10, 10]   

    entry_count_month_avg  exit_count_month_avg  total_monthly_capacity  
9               43.037619             41.303333               84.340952  
7               77.336667             76.186190              153.522857  
6               78.913810             79.128571              158.042381  
8              137.214762            127.136190              264.350952  
10             175.699524            154.287143              329.986667  
--> Monthly average total capacity: 990.2438095238094
--> Monthly average entry capacity: 512.202380952381
--> Monthly average exit capacity: 478.0414285714286
--> Top 3 capacity station in

## 2.5 Capacity for all communites

In [38]:
community_capcity_full = community_capcity_df.copy()
community_capcity_full = community_capcity_full.groupby("community").sum()
community_capcity_full = community_capcity_full.sort_values("total_monthly_capacity")
community_capcity_full

,entry_count_month_avg,exit_count_month_avg,total_monthly_capacity
community,,,
45,140.972857,139.822857,280.795714
31,172.496667,171.447143,343.943810
46,173.267619,171.179524,344.447143
2,228.001905,230.008571,458.010476
14,256.280000,244.897619,501.177619
23,258.589524,258.246190,516.835714
10,512.202381,478.041429,990.243810
29,513.484286,561.651905,1075.136190


## Find shortest Path

In [39]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [40]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [41]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [42]:
my_neo4j_wipe_out_database()

In [43]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [44]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [45]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [46]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [47]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [48]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [49]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [50]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


In [51]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
#     for r in result:
        
#         total_cost = int(r['totalCost'])
        
#         print("\n--------------------------------")
#         print("   Total Cost: ", total_cost)
#         print("   Minutes: ", round(total_cost / 60.0,1))
#         print("--------------------------------")
        
#         nodes = r['nodes']
#         costs = r['costs']
        
#         i = 0
#         previous = 0
        
#         for n in nodes:
            
#             print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
#             previous = int(costs[i])
#             i += 1
    for r in result:
        total_cost = int(r['totalCost'])
    return round(total_cost / 60.0,1)

In [79]:
delivery_station_df = community_capcity_df[['name' , 'community', 'total_monthly_capacity']].groupby(['community']).max()
from_stations = []
to_stations = []
time = []
from_station = delivery_station_df.loc[delivery_station_df['total_monthly_capacity'].idxmax()]['name']
for to_station in delivery_station_df['name'].to_list():
    if from_station != to_station:
        from_stations.append(from_station)
        to_stations.append(to_station)
        time.append(my_neo4j_shortest_path('depart ' + from_station, 'arrive ' + to_station))
delivery_df = pd.DataFrame({'From Station': from_stations, 'To Station': to_stations,
                            'Time (min)': time })
delivery_df.sort_values(['Time (min)'])

,From Station,To Station,Time (min)
1,West Oakland,Powell Street,10.0
6,West Oakland,Rockridge,14.0
4,West Oakland,Walnut Creek,29.0
0,West Oakland,Richmond,30.0
2,West Oakland,South San Francisco,32.0
3,West Oakland,West Dublin,37.0
5,West Oakland,Warm Springs,47.0
